In [1]:
import ccxt
import pandas as pd
from datetime import datetime

In [2]:
class CryptoDataCollector:
    """
    A class for collecting cryptocurrency market data and social sentiment data from various exchanges.

    Attributes:
    ----------
    exchanges : list
        A list of initialized ccxt exchange objects based on the provided exchange names.
    """

    def __init__(self, exchanges=None):
        """
        Initialize the CryptoDataCollector with a list of exchange names.

        Parameters:
        ----------
        exchanges : list, optional
            A list of exchange names (default includes exchanges that do not require API key).
        """
        if exchanges is None:
            exchanges = [
                'binanceus', 'coinbase', 'kraken', 'bitfinex', 'kucoin', 
                'okx', 'gateio', 'bitstamp', 'poloniex', 'huobi', 
                'hitbtc', 'bitget', 'exmo', 'phemex', 'upbit'
            ]
        self.exchanges = [self._initialize_exchange(ex) for ex in exchanges]

    def _initialize_exchange(self, exchange_name):
        """
        Helper method to initialize a ccxt exchange instance.

        Parameters:
        ----------
        exchange_name : str
            The name of the exchange to initialize.

        Returns:
        -------
        ccxt.Exchange
            An initialized ccxt exchange object.
        """
        try:
            return getattr(ccxt, exchange_name)()
        except AttributeError:
            raise ValueError(f"Exchange '{exchange_name}' is not supported by ccxt.")

    def fetch_historical_data(self, symbol='BTC/USDT', timeframe='5m', limit=10000):
        """
        Fetch historical OHLCV data from all configured exchanges for a given symbol and timeframe.

        Parameters:
        ----------
        symbol : str, optional
            The trading pair symbol (default is 'BTC/USDT').
        timeframe : str, optional
            The timeframe for OHLCV data (default is '5m').
        limit : int, optional
            The maximum number of data points to fetch per exchange (default is 10000, the exchanges usually give less data).

        Returns:
        -------
        pd.DataFrame
            A concatenated DataFrame containing OHLCV data from all exchanges.
        """
        data = []
        for exchange in self.exchanges:
            try:
                ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  # Convert to readable format
                df['exchange'] = exchange.id  # Add exchange identifier
                data.append(df)
            except Exception as e:
                print(f"Error fetching data from {exchange.id}: {e}")
        if not data:
            raise ValueError("No data was fetched. Check symbol, timeframe, or exchange availability.")
        return pd.concat(data, ignore_index=True)

    def fetch_social_sentiment(self, symbol):
        """
        Placeholder for fetching social sentiment data for a given symbol.
        
        Parameters:
        ----------
        symbol : str
            The trading pair or cryptocurrency symbol (e.g., 'BTC', 'ETH').
        
        Notes:
        -----
        - This method should be implemented with APIs such as Twitter, Reddit, or other sentiment analysis tools.
        - Use libraries like `tweepy`, `praw`, or web scraping to collect relevant data.
        """
        raise NotImplementedError("Social sentiment fetching is not implemented yet.")


In [3]:
btc_price=CryptoDataCollector().fetch_historical_data()

In [4]:
btc_price['timestamp'] = pd.to_datetime(btc_price['timestamp'])
btc_price.to_csv('btc_data.csv',  index=False, date_format='%Y-%m-%d %H:%M:%S')